# Artificial Intelligence
A Jupyter Notebook exploring deep learning.

In [ ]:
from __future__ import print_function

import plaidml.keras
plaidml.keras.install_backend()

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...
Epoch 1/60
200285/200285 [==============================] - 92s 457us/step - loss: 1.9933

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "e advocates of a criminal are seldom art"
e advocates of a criminal are seldom arting the self--and and the self--and the self--and the streation of the self--and the pristical the self--and the superiation of the self--and man and and the pristical to a preser the self--and the pristic of the part of the stregicts the philosophers and the self--and the present the self--and for a sure the serst of the pristical the streation of the pristic of the somet the self--and the pristi
----- diversity: 0.5
----- Generating with seed: "e advocates of a criminal are seldom art"
e advocates of a criminal are seldom arting as the sidlifed to instinct of in the streed superticiss that the the partion of the somen to the persons and an

injustice; schumann, with his wame effected, politicanately a blidation of sort, as he
has intentations our life own appresinge memand or else ray his swrenged
to beenwise exerching presentain of the
philosophy; the
doet such himself this new vight
diened who ising evircy, always be only other of
feeling, this velver in the uncrimated the peoples, for things and corpleboly powerful, by who immece a carret and ential thoughter 
----- diversity: 1.2
----- Generating with seed: "extent of
injustice; schumann, with his "
extent of
injustice; schumann, with his depess opber-saye, young, it is eardence--no "the aimes a regre, thoughs,--and "offoken, but recomstinglys in
well
obmers, out nok lew nature act, but they sufponings, as the orden
us
cifming question.

leatod wouldpocomakes, the whole that man worth
araint quite anythirg paop; doe--here
oscivelnes as thereofii al religancewovation and theughous. with the sundne, proctly bestimaled thouth, (in her
Epoch 5/60
148992/200285 [==========